In [99]:
!pip install pyspark
!pip install findspark

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [100]:
import re

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StructType,StructField, StringType, \
IntegerType,BooleanType,DoubleType
from pyspark import SparkContext, HiveContext,SQLContext

In [101]:
spark = SparkSession.builder.getOrCreate()

In [102]:
path = "/kaggle/input/movies-datas/"
df = spark.read.option("multiline","true").json(path + "*.json")
df = df.withColumn("file_name", F.input_file_name())

We will transpose the dataframe and change the name of each column by the name of the movie

In [103]:
def extract_file_name(cell_value):
    if isinstance(cell_value, str): 
        match = re.search(r'/([^/]+)\.json$', cell_value)
        if match:
            return match.group(1)
    return cell_value

df_pandas = df.toPandas()
df_pandas_T = df_pandas.T
df_pandas_T.reset_index(drop=True, inplace=True)

for col_name in df_pandas_T.columns:
    df_pandas_T[col_name] = df_pandas_T[col_name].apply(extract_file_name)

In [104]:
last_values = df_pandas_T.apply(lambda col: col.iloc[-1])
dfP_T.columns = last_values

now, we are switching from the pandas format to Spark

In [105]:
spark_df = spark.createDataFrame(df_pandas_T)

In [106]:
def modify_columns_name(df, actual_name, desired_name):
    for old_name in spark_df.columns:
        new_name = old_name.replace(actual_name, desired_name)
        df = df.withColumnRenamed(old_name, new_name)
    
    df = df.limit(df.count() - 1)
    return df

spark_df = modify_columns_name(spark_df, "%20","_")
spark_df = modify_columns_name(spark_df, "commentaires_","")
spark_df = modify_columns_name(spark_df, ".","")

23/12/23 22:16:17 WARN TaskSetManager: Stage 43 contains a task of very large size (3451 KiB). The maximum recommended task size is 1000 KiB.
23/12/23 22:16:23 WARN TaskSetManager: Stage 46 contains a task of very large size (3451 KiB). The maximum recommended task size is 1000 KiB.
23/12/23 22:16:31 WARN TaskSetManager: Stage 49 contains a task of very large size (3451 KiB). The maximum recommended task size is 1000 KiB.


We will drop the column with only null values

In [107]:
def drop_null_columns(df): 

    null_counts = df.select([F.count(F.when(F.col(c).isNull(), c)).alias( 
        c) for c in df.columns]).collect()[0].asDict()
    col_to_drop = [k for k, v in null_counts.items() if v > 0]
    df = df.drop(*col_to_drop)

    return df

spark_df = drop_null_columns(spark_df)

23/12/23 22:16:33 WARN TaskSetManager: Stage 52 contains a task of very large size (3451 KiB). The maximum recommended task size is 1000 KiB.


Now, we save the datas on a parquet file

In [108]:
spark_df.write.mode("overwrite").parquet("avis_films_imd")

23/12/23 22:16:34 WARN TaskSetManager: Stage 55 contains a task of very large size (3451 KiB). The maximum recommended task size is 1000 KiB.
